# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [7]:
from datetime import datetime as dt
from SPARQLWrapper import SPARQLWrapper, JSON

## Obtenir la liste des footballeurs belges

In [14]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
sparql = SPARQLWrapper(endpoint)

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death
statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .           # Nationalité : belge
    ?person wdt:P106 wd:Q937857 .      # Profession : footballeur
    ?person wdt:P569 ?dateBirth .      # Date de naissance
    OPTIONAL {?person wdt:P570 ?dateDeath .}  # Date de décès (si elle existe)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

rows = results['results']['bindings']
print(f"\n{len(rows)} Belgian footballers found\n")
print(rows[:])


3676 Belgian footballers found

[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28811930'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Aaron Dhondt'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1995-12-18T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28811930'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Aaron Dhondt'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1995-12-19T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q18608500'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Aaron Leya Iseka'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1997-11-15T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q16234523'}, 'personLabel': {'xml:lang'

## Filtrer pour n'afficher que les noms contenant "Albert"

In [16]:
name_filter = 'Albert'
max_results = 1000

date_format = "%Y-%m-%dT%H:%M:%SZ"
filtered_rows = [row for row in rows if name_filter in row['personLabel']['value']]
print(f"Displaying the first {max_results}:\n")
for row in filtered_rows[:max_results]:
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format)
        birth_year = birth_date.year
    except ValueError:
        birth_year = "????"
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format)
        death_year = death_date.year
    except ValueError: # unknown death date
        death_year = "????"
    except KeyError: # still alive
        death_year = ""
    print(f"{row['personLabel']['value']} ({birth_year}-{death_year})")

Displaying the first 1000:

Jude Vandelannoite (1973-)


## Pour en savoir plus

- [Le projet Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page)
- [Aide à la construction de requêtes](https://query.wikidata.org/)